<a href="https://colab.research.google.com/github/IlyaKo4atkov/MyProjects/blob/main/Quiz_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Устанавливаем необходимые библиотеки
!pip install aiogram
!pip install aiosqlite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.7/575.7 kB 3.1 MB/s eta 0:00:00


In [ ]:
#Установка библиотеки nest_asyncio
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:

import aiosqlite
import asyncio
import logging
from aiogram import Bot, Dispatcher, types
from aiogram.filters.command import Command
from aiogram.utils.keyboard import InlineKeyboardBuilder, ReplyKeyboardBuilder
from aiogram import F

# Включаем логирование, чтобы не пропустить важные сообщения
logging.basicConfig(level=logging.INFO)

#Токен моего бота, полученный от BotFather
API_TOKEN = '7225654963:AAEbWtV_-iWlu5xtuq45vITV4ccJh95SVXU'

# Объект бота
bot = Bot(token=API_TOKEN)
# Диспетчер
dp = Dispatcher()

# Зададим имя базы данных
DB_NAME = 'quiz_bot.db'


# Структура квиза
quiz_data = [
    {
        'question': 'Что такое Python?',
        'options': ['Язык программирования', 'Тип данных', 'Музыкальный инструмент', 'Змея на английском'],
        'correct_option': 0
    },
    {
        'question': 'Какой тип данных используется для хранения целых чисел?',
        'options': ['int', 'float', 'str', 'natural'],
        'correct_option': 0
    },
    {
        'question': 'В каком месяце 28 дней?',
        'options': ['В каждом', 'Январь', 'Февраль', 'Июнь', 'Июль'],
        'correct_option': 0
    },
    {
        'question': 'Какого объекта нет на рабочем столе компьютера?',
        'options': ['Панель управления', "Панель задач", "Корзина", "Сетевое окружение"],
        'correct_option': 0
    },
    {
        'question': 'Какая единица измерения не относится к измерению инфомации?',
        'options': ['герц', 'бит', 'байт', 'бод'],
        'correct_option': 0
    },
    {
        'question': 'Кто является основателем компании Microsoft?',
        'options': ['Билл Гейтс', 'Марк Цукерберг', 'Билл Клинтон', 'Стив Джобс'],
        'correct_option': 0
    },
    {
        'question': 'Какого свойства информации не существует?',
        'options': ['Турбулентность', 'Дискретность', 'Результативность', 'Детерминированность'],
        'correct_option': 0
    },
    {
        'question': 'Программа, управляющая работой внешних устройств называется',
        'options': ['Драйвер', 'Компилятор', 'Архиватор', 'Модулятор'],
        'correct_option': 0
    },
    {
        'question': 'Сколько языков программирования существует в наше время?',
        'options': ['Несколько тысяч', 'Один', '293', '550'],
        'correct_option': 0
    },
    {
        'question': 'Как называют специалистов, разрабатывающих программное обеспечение?',
        'options': ['Программистами', 'Хакерами', 'Верстальщиками', 'Системный администратор'],
        'correct_option': 0
    }]

#Глобальная переменная для подсчета правильных ответов
correct_answers_count = 0
#Функция создания колбэк-клавиатуры с вариантами ответов
def generate_options_keyboard(answer_options, right_answer):
  #Создание сборщика клавиатур типа Inline
    builder = InlineKeyboardBuilder()
    #Создание 4 Inline кнопки, которые равны количеству ответов
    for option in answer_options:
        builder.add(types.InlineKeyboardButton(
            text=option,
            #Присвоим данные для колбэк запроса.
            #Если ответ верный сформируется колбэк-запрос с данными "right_answer"
            #Если ответ неверный сформируется колбэк-запрос с данными "wrong_answer"
            callback_data="right_answer" if option == right_answer else "wrong_answer")
        )
    #Производится вывод по одной кнопке в столбик
    builder.adjust(1)
    return builder.as_markup()

#создание перехватчика колбэк запросов
@dp.callback_query(F.data == "right_answer")
async def right_answer(callback: types.CallbackQuery):
    global correct_answers_count
  #Редактирование текущего сообщения с целью убрвть кнопки(reply_markup=None)
    await callback.bot.edit_message_reply_markup(
        chat_id=callback.from_user.id,
        message_id=callback.message.message_id,
        reply_markup=None
    )
    #Отправляем в чат сообщение что отввет верный
    await callback.message.answer("Это правильный ответ!")
    correct_answers_count += 1
    #Получение текущего вопроса для данного пользователя
    current_question_index = await get_quiz_index(callback.from_user.id)
    # Обновление номера текущего вопроса в базе данных
    current_question_index += 1
    await update_quiz_index(callback.from_user.id, current_question_index)
    #Проверка достигнут ли конец вопроса
    if current_question_index < len(quiz_data):
        await get_question(callback.message, callback.from_user.id)
    else:
        await callback.message.answer("Это был последний вопрос. Викторина завершена!")
        await callback.message.answer(f'Количество правильных ответов {correct_answers_count} из {len(quiz_data)}')
        #Обуляем счетчик, для дальнейшей корректной работы бота. В случае если не произвести обноление, то при повторном старте бота, счетчик будет суммировать общие правильные вопросы
        correct_answers_count = 0


@dp.callback_query(F.data == "wrong_answer")
async def wrong_answer(callback: types.CallbackQuery):
    global correct_answers_count
  #Редактировние текущего сообщения с целью убрать кнопки (reply_markup=None)
    await callback.bot.edit_message_reply_markup(
        chat_id=callback.from_user.id,
        message_id=callback.message.message_id,
        reply_markup=None
    )

    # Получение текущего вопроса из словаря состояний пользователя
    current_question_index = await get_quiz_index(callback.from_user.id)
    correct_option = quiz_data[current_question_index]['correct_option']
    #Отправка в чат сообщения об указании ошибочного ответа
    await callback.message.answer(f"Неправильно. Правильный ответ: {quiz_data[current_question_index]['options'][correct_option]}")


    # Обновление номера текущего вопроса в базе данных
    current_question_index += 1
    await update_quiz_index(callback.from_user.id, current_question_index)


    if current_question_index < len(quiz_data):
        await get_question(callback.message, callback.from_user.id)
    else:
        await callback.message.answer("Это был последний вопрос. Викторина завершена!")
        await callback.message.answer(f'Количество правильных ответов {correct_answers_count} из {len(quiz_data)}')
        #Обуляем счетчик, для дальнейшей корректной работы бота. В случае если не произвести обноление, то при повторном старте бота, счетчик будет суммировать общие правильные вопросы
        correct_answers_count = 0


# Хэндлер на команду /start
@dp.message(Command("start"))
async def cmd_start(message: types.Message):
    builder = ReplyKeyboardBuilder()
    builder.add(types.KeyboardButton(text="Начать игру"))
    await message.answer("Добро пожаловать в викторину!", reply_markup=builder.as_markup(resize_keyboard=True))


async def get_question(message, user_id):

    # Получение текущего вопроса из словаря состояний пользователя
    current_question_index = await get_quiz_index(user_id)
    #Получаем индекс правильного ответа для текущего вопроса
    correct_index = quiz_data[current_question_index]['correct_option']
    #Получаем список вариантов ответа для текущего вопроса
    opts = quiz_data[current_question_index]['options']
    #Функция генерации кнопок для текущего вопроса викторины
    #В качестве аргументов передаем варианты ответов и значение правильного ответа
    kb = generate_options_keyboard(opts, opts[correct_index])
    #Отправляем в чат сообщение с вопросом, прикрепляем сгенерированные кнопки
    await message.answer(f"{quiz_data[current_question_index]['question']}", reply_markup=kb)


async def new_quiz(message):
  #Получаем id пользователя, отправившего сообщение
    user_id = message.from_user.id
    #сбрасываем значение текущего индекса вопроса викторины в 0
    current_question_index = 0
    await update_quiz_index(user_id, current_question_index)
    #Запрос нового вопроса для викторины
    await get_question(message, user_id)


async def get_quiz_index(user_id):
     # Подключаемся к базе данных
     async with aiosqlite.connect(DB_NAME) as db:
        # Получаем запись для заданного пользователя
        async with db.execute('SELECT question_index FROM quiz_state WHERE user_id = (?)', (user_id, )) as cursor:
            # Возвращаем результат
            results = await cursor.fetchone()
            if results is not None:
                return results[0]
            else:
                return 0


async def update_quiz_index(user_id, index):
    # Создаем соединение с базой данных (если она не существует, она будет создана)
    async with aiosqlite.connect(DB_NAME) as db:
        # Вставляем новую запись или заменяем ее, если с данным user_id уже существует
        await db.execute('INSERT OR REPLACE INTO quiz_state (user_id, question_index) VALUES (?, ?)', (user_id, index))
        # Сохраняем изменения
        await db.commit()


# Хэндлер на команду /quiz
@dp.message(F.text=="Начать игру")
@dp.message(Command("quiz"))
async def cmd_quiz(message: types.Message):

    await message.answer(f"Давайте начнем викторину!")
    await new_quiz(message)



async def create_table():
    # Создаем соединение с базой данных (если она не существует, она будет создана)
    async with aiosqlite.connect(DB_NAME) as db:
        # Создаем таблицу
        await db.execute('''CREATE TABLE IF NOT EXISTS quiz_state (user_id INTEGER PRIMARY KEY, question_index INTEGER)''')
        # Сохраняем изменения
        await db.commit()

# Запуск процесса поллинга новых апдейтов
async def main():

    # Запускаем создание таблицы базы данных
    await create_table()

    await dp.start_polling(bot)

if __name__ == "__main__":
    asyncio.run(main())